In [1]:
import ee
import geemap
import geopandas as gpd
import pygris
from pygris import states
import rasterio
import fiona

In [2]:
# ee.Authenticate()
ee.Initialize(project='hotspotstoplight')


Successfully saved authorization token.


In [3]:
us = states(cb = True, resolution = "20m")
nj = us[us['NAME'] == 'New Jersey']
nj_ee = geemap.geopandas_to_ee(nj)

# Create a Map
Map = geemap.Map()

Map.addLayerControl()

# Filter the NLCD Tree Canopy Cover (TCC) for 2021 and CONUS
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2021_REL/TCC/v2021-4')
tcc = dataset.filter(ee.Filter.calendarRange(2021, 2021, 'year')) \
             .filter(ee.Filter.eq('study_area', 'CONUS')) \
             .first() \
             .clip(nj_ee)  # Clip to NJ boundaries

# Add TCC layer
Map.addLayer(tcc.select('NLCD_Percent_Tree_Canopy_Cover'), {'min': 0, 'max': 60, 'palette': tcc_palette}, 'NLCD Percent Tree Canopy Cover')

# World Settlement Footprint (WSF) 2015 clipped to NJ
wsf_dataset = ee.Image('DLR/WSF/WSF2015/v1').clip(nj_ee)
Map.addLayer(wsf_dataset, wsf_vis, 'Human settlement areas')

# Population Density clipped to NJ
population_dataset = ee.ImageCollection('CIESIN/GPWv411/GPW_Population_Density').first().clip(nj_ee)
Map.addLayer(population_dataset, population_vis, 'Population Density')

# Global Tidal Wetland Change 2019 clipped to NJ
tidal_dataset = ee.Image('JCU/Murray/GIC/global_tidal_wetland_change/2019').clip(nj_ee)
Map.addLayer(tidal_dataset.select('loss'), {'palette': loss_palette, 'min': 1, 'max': 1}, 'Tidal Wetland Loss')
Map.addLayer(tidal_dataset.select('gain'), {'palette': gain_palette, 'min': 1, 'max': 1}, 'Tidal Wetland Gain')

# Bio-Intactness Dataset clipped to NJ
bdi_ic = ee.ImageCollection("projects/ebx-data/assets/earthblox/IO/BIOINTACT").mean().clip(nj_ee)
Map.addLayer(bdi_ic, bdi_vis, "Bio-Intactness")

# Biodiversity, Carbon, and Water datasets clipped to NJ
biodiv_biome_clipped = biodiv_biome.clip(nj_ee)
biodivcarbon_clipped = biodivcarbon.clip(nj_ee)
biodivcarbonwater_clipped = biodivcarbonwater.clip(nj_ee)

# Add Biodiversity and Carbon layers
Map.addLayer(biodivcarbon_clipped, biodiv_carbon_vis, "Biodiversity and Carbon")
Map.addLayer(biodivcarbonwater_clipped, biodiv_carbon_vis, "Biodiversity, Carbon, and Water")

# Global Forest Change dataset clipped to NJ
gfc_dataset = ee.Image('UMD/hansen/global_forest_change_2022_v1_10').clip(nj_ee)
Map.addLayer(gfc_dataset, treeCoverVisParam, 'Tree Cover')
Map.addLayer(gfc_dataset, treeLossVisParam, 'Tree Loss Year')

# Set the map center to New Jersey with an appropriate zoom level
Map.setCenter(-74.4057, 40.0583, 7)  # Longitude, Latitude, Zoom Level

# Show the map
Map

In [4]:
def export_to_drive(image, description, folder, region, scale=250):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=description,
        folder=folder,
        scale=scale,
        region=region.geometry().bounds().getInfo()['coordinates'],
        fileFormat='GeoTIFF'
    )
    task.start()

# Assuming 'nj_ee' is your Earth Engine geometry for New Jersey
# Assuming each dataset is clipped to 'nj_ee' and ready for export

# Dictionary of datasets and their names for export
datasets = {
    'TCC_NJ': tcc.select('NLCD_Percent_Tree_Canopy_Cover'),
    'WSF2015_NJ': wsf_dataset,
    'PopulationDensity_NJ': population_dataset,
    'TidalWetlandChange2019_NJ': tidal_dataset,
    # Add more datasets here as needed
}

# Folder name in Google Drive
folder_name = "MACH Studio Data"

# Iterate over the datasets dictionary and export each
for description, image in datasets.items():
    export_to_drive(image, description, folder_name, nj_ee)

print("Export tasks started. Check the 'Tasks' tab in the Google Earth Engine Code Editor for progress.")